In [ ]:
from movie_data_scraper import TMDB_Scraper
import pandas as pd

In [ ]:
tmdb_scraper = TMDB_Scraper(api_key="abd17a9f250807b76ebbfa9997ca6ade")

Do a preliminary search of the results we want to aggregate using tMDb discover API, storing the total number of pages so we can aggregate all of them by iterating later.

In [9]:
discover_query_results = tmdb_scraper.run_api("discover/movie", js_query_args={
    'sort_by': 'vote_average.desc',
    'vote_count.gte': 2000
})
num_pages = discover_query_results['total_pages']

Make Blank DataFrame to append results to

In [10]:
all_trailers = pd.DataFrame({"movie_id": [], 'movie_title': [], "title": [], "youtube_key": []})

Iterate through and aggregate all results

In [11]:
for p in range(1, num_pages+1):
    this_page_results = tmdb_scraper.run_api("discover/movie", js_query_args={
        'sort_by': 'vote_average.desc',
        'vote_count.gte': 2000,
        'page': p
    })
    print(f"Aggregating page {p} of {num_pages}")
    for i, movie in enumerate(this_page_results['results']):
        print(f"Aggregating trailers of movie {i} of 20")
        videos = tmdb_scraper.movie_videos(movie['id'])
        trailers = [v for v in videos['results'] if v['type'] == 'Trailer']
        for t in trailers:
            all_trailers = all_trailers.append({
                "movie_id": movie['id'],
                'movie_title': movie['title'],
                'title': t['name'],
                'youtube_key': t['key']
            }, ignore_index=True)

        all_trailers.to_csv("trailers.csv")

Aggregating page 1 of 60
Aggregating trailers of movie 0 of 20
Aggregating trailers of movie 1 of 20
Aggregating trailers of movie 2 of 20
Aggregating trailers of movie 3 of 20
Aggregating trailers of movie 4 of 20
Aggregating trailers of movie 5 of 20
Aggregating trailers of movie 6 of 20
Aggregating trailers of movie 7 of 20
Aggregating trailers of movie 8 of 20
Aggregating trailers of movie 9 of 20
Aggregating trailers of movie 10 of 20
Aggregating trailers of movie 11 of 20
Aggregating trailers of movie 12 of 20
Aggregating trailers of movie 13 of 20
Aggregating trailers of movie 14 of 20
Aggregating trailers of movie 15 of 20
Aggregating trailers of movie 16 of 20
Aggregating trailers of movie 17 of 20
Aggregating trailers of movie 18 of 20
Aggregating trailers of movie 19 of 20
Aggregating page 2 of 60
Aggregating trailers of movie 0 of 20
Aggregating trailers of movie 1 of 20
Aggregating trailers of movie 2 of 20
Aggregating trailers of movie 3 of 20
Aggregating trailers of movi

Aggregating trailers of movie 6 of 20
Aggregating trailers of movie 7 of 20
Aggregating trailers of movie 8 of 20
Aggregating trailers of movie 9 of 20
Aggregating trailers of movie 10 of 20
Aggregating trailers of movie 11 of 20
Aggregating trailers of movie 12 of 20
Aggregating trailers of movie 13 of 20
Aggregating trailers of movie 14 of 20
Aggregating trailers of movie 15 of 20
Aggregating trailers of movie 16 of 20
Aggregating trailers of movie 17 of 20
Aggregating trailers of movie 18 of 20
Aggregating trailers of movie 19 of 20
Aggregating page 12 of 60
Aggregating trailers of movie 0 of 20
Aggregating trailers of movie 1 of 20
Aggregating trailers of movie 2 of 20
Aggregating trailers of movie 3 of 20
Aggregating trailers of movie 4 of 20
Aggregating trailers of movie 5 of 20
Aggregating trailers of movie 6 of 20
Aggregating trailers of movie 7 of 20
Aggregating trailers of movie 8 of 20
Aggregating trailers of movie 9 of 20
Aggregating trailers of movie 10 of 20
Aggregating t

Aggregating trailers of movie 13 of 20
Aggregating trailers of movie 14 of 20
Aggregating trailers of movie 15 of 20
Aggregating trailers of movie 16 of 20
Aggregating trailers of movie 17 of 20
Aggregating trailers of movie 18 of 20
Aggregating trailers of movie 19 of 20
Aggregating page 22 of 60
Aggregating trailers of movie 0 of 20
Aggregating trailers of movie 1 of 20
Aggregating trailers of movie 2 of 20
Aggregating trailers of movie 3 of 20
Aggregating trailers of movie 4 of 20
Aggregating trailers of movie 5 of 20
Aggregating trailers of movie 6 of 20
Aggregating trailers of movie 7 of 20
Aggregating trailers of movie 8 of 20
Aggregating trailers of movie 9 of 20
Aggregating trailers of movie 10 of 20
Aggregating trailers of movie 11 of 20
Aggregating trailers of movie 12 of 20
Aggregating trailers of movie 13 of 20
Aggregating trailers of movie 14 of 20
Aggregating trailers of movie 15 of 20
Aggregating trailers of movie 16 of 20
Aggregating trailers of movie 17 of 20
Aggregati

Aggregating trailers of movie 19 of 20
Aggregating page 32 of 60
Aggregating trailers of movie 0 of 20
Aggregating trailers of movie 1 of 20
Aggregating trailers of movie 2 of 20
Aggregating trailers of movie 3 of 20
Aggregating trailers of movie 4 of 20
Aggregating trailers of movie 5 of 20
Aggregating trailers of movie 6 of 20
Aggregating trailers of movie 7 of 20
Aggregating trailers of movie 8 of 20
Aggregating trailers of movie 9 of 20
Aggregating trailers of movie 10 of 20
Aggregating trailers of movie 11 of 20
Aggregating trailers of movie 12 of 20
Aggregating trailers of movie 13 of 20
Aggregating trailers of movie 14 of 20
Aggregating trailers of movie 15 of 20
Aggregating trailers of movie 16 of 20
Aggregating trailers of movie 17 of 20
Aggregating trailers of movie 18 of 20
Aggregating trailers of movie 19 of 20
Aggregating page 33 of 60
Aggregating trailers of movie 0 of 20
Aggregating trailers of movie 1 of 20
Aggregating trailers of movie 2 of 20
Aggregating trailers of m

KeyboardInterrupt: 